# CLUSTERING - RANDOM FOREST

# Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing, cross_validation

C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\madhu\Anaconda3\lib\site-packages\sklearn\lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\madhu\Anacon

## Load clean loan csv 

In [3]:
loan_df = pd.read_csv('../Loan.csv', low_memory=False) 

In [ ]:
loan_df.columns

In [5]:
loan_df['policy_code'] = loan_df['policy_code'].astype(int)

# Remove General Columns Not Needed

In [5]:
feature_df = loan_df[['application_type','emp_length', 'dti', 'annual_inc',
       'int_rate', 'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico', 'approval']]

# Dealing with catagorical variables

In [6]:
home_ownerships = pd.get_dummies(feature_df['home_ownership'], prefix='house')
feature_df = feature_df.join(home_ownerships)
feature_df.drop('home_ownership', axis=1, inplace=True)

app_type = pd.get_dummies(feature_df['application_type'], prefix='appType')
feature_df = feature_df.join(app_type)
feature_df.drop('application_type', axis=1, inplace=True)

adr_states = pd.get_dummies(feature_df['State'], prefix='s')
feature_df = feature_df.join(adr_states)
feature_df.drop('State', axis=1, inplace=True)

#purpose   
purpose = pd.get_dummies(feature_df['purpose'], prefix='purpose')
feature_df = feature_df.join(purpose)
feature_df.drop('purpose', axis=1, inplace=True)

# No Clustering

In [53]:
zero_cluster_df = feature_df

In [54]:
X_train, X_test, y_train, y_test = train_test_split(zero_cluster_df.ix[:, zero_cluster_df.columns != 'int_rate'], 
                                            zero_cluster_df['int_rate'], 
                                            test_size=0.30)

In [55]:
rf = RandomForestRegressor(n_estimators=60)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [56]:
# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

0.91570957196


In [57]:
y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

In [59]:
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

-----Train-----
RMS:  1.3653917422006892
MAE:  1.02942860799
MAPE:  8.388974819000946
-----Test-----
RMS:  3.621423338283415
MAE:  2.76597611817
MAPE:  22.567188962954894


# Manual Clustering

In [60]:
manual_df = feature_df
def clustering(fico):
    cluster_name = ''
    if fico>790.0:
        cluster_name ='K1'
    elif ((fico <= 790.0) & (fico>750.0)):
        cluster_name = 'K2'
    elif ((fico <=750.0) & (fico>700.0)):
        cluster_name ='K3'
    elif ((fico <=700.0) & (fico>660.0)):
        cluster_name= 'K4'
    return cluster_name
manual_df['cluster'] = manual_df['fico'].astype(float).map(lambda x: clustering(x))

In [61]:
manual_df.head()

,emp_length,dti,annual_inc,int_rate,loan_amnt,term,fico,approval,house_ANY,house_MORTGAGE,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,cluster
0,10,27.65,24000.0,10.65,5000.0,36,737.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
1,1,1.00,30000.0,15.27,2500.0,60,742.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
2,10,8.72,12252.0,15.96,2400.0,36,737.0,1,0,0,...,0,0,0,0,0,0,1,0,0,K3
3,10,20.00,49200.0,13.49,10000.0,36,692.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4
4,1,17.94,80000.0,12.69,3000.0,60,697.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4


In [62]:
#Seperate into diff dataframes
manual_c1 = manual_df[manual_df['cluster']=='K1']
manual_c1.drop('cluster',axis=1,inplace=True)
manual_c2 = manual_df[manual_df['cluster']=='K2']
manual_c2.drop('cluster',axis=1,inplace=True)
manual_c3 = manual_df[manual_df['cluster']=='K3']
manual_c3.drop('cluster',axis=1,inplace=True)
manual_c4 = manual_df[manual_df['cluster']=='K4']
manual_c4.drop('cluster',axis=1,inplace=True)

In [63]:
#Manual 1
X_train, X_test, y_train, y_test = train_test_split(manual_c1.ix[:, manual_c1.columns != 'int_rate'], 
                                            manual_c1['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train))
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test))
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.911108546776
-----Train-----
RMS:  0.9417753517711168
MAE:  0.635550693847
MAPE:  0.07504401234926672
-----Test-----
RMS:  2.4391171684956174
MAE:  1.68122858885
MAPE:  0.1983450672983454


In [33]:
#manual 2
X_train, X_test, y_train, y_test = train_test_split(manual_c2.ix[:, manual_c2.columns != 'int_rate'], 
                                            manual_c2['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)



#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train))
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.906278490354
-----Train-----
RMS:  1.1164585319551865
MAE:  0.787638460991
MAPE:  8.58870617074232
-----Test-----
RMS:  2.9401312677168545
MAE:  2.10222240015
MAPE:  22.953272156575824


In [64]:
#Manual 3
X_train, X_test, y_train, y_test = train_test_split(manual_c3.ix[:, manual_c3.columns != 'int_rate'], 
                                            manual_c3['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train))
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test))
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.906580656282
-----Train-----
RMS:  1.3074766892289085
MAE:  0.982107592976
MAPE:  0.08983705909746303
-----Test-----
RMS:  3.449707412935476
MAE:  2.62498282121
MAPE:  0.24078770827111284


In [65]:
#Manual 4
X_train, X_test, y_train, y_test = train_test_split(manual_c4.ix[:, manual_c4.columns != 'int_rate'], 
                                            manual_c4['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.899716618317
-----Train-----
RMS:  1.4246667404404396
MAE:  1.08803070645
MAPE:  8.094641623609645
-----Test-----
RMS:  3.7824351753660186
MAE:  2.92358161704
MAPE:  21.765301258555095


# K-Means

In [7]:
k_cluster = joblib.load(open('../kmeanCluster.pkl', 'rb'))
#Create new column in load_df for the k-means clustering 
k_mean_feature_df = feature_df
k_mean_feature_df['kMean'] = k_cluster.labels_

In [8]:
kmean_0 = k_mean_feature_df[k_mean_feature_df['kMean']==0]
kmean_0.drop('kMean', axis=1, inplace=True)
kmean_1 = k_mean_feature_df[k_mean_feature_df['kMean']==1]
kmean_1.drop('kMean', axis=1, inplace=True)
kmean_2 = k_mean_feature_df[k_mean_feature_df['kMean']==2]
kmean_2.drop('kMean', axis=1, inplace=True)
kmean_3 = k_mean_feature_df[k_mean_feature_df['kMean']==3]
kmean_3.drop('kMean', axis=1, inplace=True)
kmean_4 = k_mean_feature_df[k_mean_feature_df['kMean']==4]
kmean_4.drop('kMean', axis=1, inplace=True)
kmean_5 = k_mean_feature_df[k_mean_feature_df['kMean']==5]
kmean_5.drop('kMean', axis=1, inplace=True)
kmean_6 = k_mean_feature_df[k_mean_feature_df['kMean']==6]
kmean_6.drop('kMean', axis=1, inplace=True)
kmean_7 = k_mean_feature_df[k_mean_feature_df['kMean']==7]
kmean_7.drop('kMean', axis=1, inplace=True)


In [9]:
#K-mean_0
X_train, X_test, y_train, y_test = train_test_split(kmean_0.ix[:, kmean_0.columns != 'int_rate'], 
                                            kmean_0['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train))
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) 
("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.908327968608
-----K-MEANS-----
-----Train-----
RMS:  1.3094598478734147
MAE:  0.986069852759
MAPE:  0.08529245819527063
RMS:  3.463937531434099
MAE:  2.6361188782
MAPE:  0.2274995545499195


In [10]:
#K-mean_1
X_train, X_test, y_train, y_test = train_test_split(kmean_1.ix[:, kmean_1.columns != 'int_rate'], 
                                            kmean_1['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train))
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test))
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.908262504958
-----K-MEANS-----
-----Train-----
RMS:  1.4975024100421304
MAE:  1.13987558207
MAPE:  0.08135625398780764
-----Test-----
RMS:  3.931813418317967
MAE:  3.04365988444
MAPE:  0.21745199748228294


In [11]:
#K-mean_2
X_train, X_test, y_train, y_test = train_test_split(kmean_2.ix[:, kmean_2.columns != 'int_rate'], 
                                            kmean_2['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.900780181718
-----K-MEANS-----
-----Train-----
RMS:  1.3314900975708837
MAE:  1.00283947481
MAPE:  8.92516099403678
-----Test-----
RMS:  3.5412051254384407
MAE:  2.69665760169
MAPE:  23.928242322003737


In [12]:
#K-mean_3
X_train, X_test, y_train, y_test = train_test_split(kmean_3.ix[:, kmean_3.columns != 'int_rate'], 
                                            kmean_3['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.909420660257
-----K-MEANS-----
-----Train-----
RMS:  1.3707070995775779
MAE:  1.03130390647
MAPE:  8.428978928278681
-----Test-----
RMS:  3.5919265078716522
MAE:  2.7483461458
MAPE:  22.50828262463927


In [13]:
#K-mean_4
X_train, X_test, y_train, y_test = train_test_split(kmean_4.ix[:, kmean_4.columns != 'int_rate'], 
                                            kmean_4['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.89413924155
-----K-MEANS-----
-----Train-----
RMS:  1.7782555058244858
MAE:  1.32322823296
MAPE:  8.695658376828993
-----Test-----
RMS:  4.410859750250901
MAE:  3.34692647059
MAPE:  22.68643250510615


In [14]:
#K-mean_5
X_train, X_test, y_train, y_test = train_test_split(kmean_5.ix[:, kmean_5.columns != 'int_rate'], 
                                            kmean_5['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.909468044981
-----K-MEANS-----
-----Train-----
RMS:  1.4569316914903374
MAE:  1.10615380015
MAPE:  8.361927051162844
-----Test-----
RMS:  3.8453134928435926
MAE:  2.95394315494
MAPE:  22.259663077444173


In [15]:
#K-mean_6
X_train, X_test, y_train, y_test = train_test_split(kmean_6.ix[:, kmean_6.columns != 'int_rate'], 
                                            kmean_6['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.905673884905
-----K-MEANS-----
-----Train-----
RMS:  1.4987490791974232
MAE:  1.1406205581
MAPE:  7.827577071323651
-----Test-----
RMS:  3.943172661873083
MAE:  3.03249012743
MAPE:  20.912258276141017


In [16]:
#K-mean_7
X_train, X_test, y_train, y_test = train_test_split(kmean_7.ix[:, kmean_7.columns != 'int_rate'], 
                                            kmean_7['int_rate'], 
                                            test_size=0.30)
rf = RandomForestRegressor(n_estimators=60)#max_depth =6,random_state=0)
rf.fit(X_train, y_train)

# Accuracy of the model created
accuracy = rf.score(X_train, y_train)
print(accuracy)

y_test_predicted = rf.predict(X_test)
y_train_predicted= rf.predict(X_train)

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print('-----K-MEANS-----')
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.905848705016
-----K-MEANS-----
-----Train-----
RMS:  1.2770719114307536
MAE:  0.960040762355
MAPE:  8.643726122137355
-----Test-----
RMS:  3.3653025647178745
MAE:  2.57188898187
MAPE:  23.33568112630891


The text in the document by SHREYANTH REDDY BEZAWADA is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

Copyright 2018 SHREYANTH REDDY BEZAWADA AND ADITYA MOHAN KUMAR

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.